Last Modified: March 15th, 2023 [OI]


# CR 2049 MCMC results (emcee hammer)

In [2]:
import numpy as np 
import copy
import scipy.optimize as opt
import emcee
import corner
import h5py

from model_chain import run_chain_of_models_mcmc
from sunpy.coordinates.sun import carrington_rotation_time
from model_chain import run_chain_of_models, get_ace_date
from MCMC_seven_params import *
import sunpy.map
import astropy.units as u
import os

In [3]:
import matplotlib.pyplot as plt
import matplotlib
font = {'family' : 'serif',
        'size'   : 11}

matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=11) 
matplotlib.rc('ytick', labelsize=11) 

* As *a posteriori* parameter dimensionality reduction we set the non-influential parameters to their nomial values, i.e.
$$\delta = 1.75, \psi = 3.5, \alpha_{acc} = 0.15, r_h = 50.$$

## Bayesian Inference and Markov Chain Monte Carlo for the Ambient Solar Wind Speed

In this problem, we use Bayesian inference and Markov chain Monte Carlo (MCMC) to find the parameters of
a chain of models that are used in space weather operational forecasting. The model and data are connected by 
$$
\mathbf{y} = \mathcal{M}(\mathbf{\theta}) + \mathbf{\eta}
$$
where $\mathbf{\theta} = \{r_{SS}, v_{0}, v_{1}, \alpha, \beta, w, \gamma\} \in \mathbb{R}^{7}$ is the set of uncertain parameters, $\mathcal{M}$ is the chain of *PFSS-WSA-HUX* models, $\mathbf{\eta}$ represents the discrepancies between the model and data, and is typically assumed to be Gaussian distributed with mean zero and covariance matrix $\mathbf{\Sigma}$ (positive definite), and $\mathbf{y}$ is the the radial velocity in 1hr cadance interval at L1. The radial velocity at L1 is meassured via SWEPAM monior on ACE. 

* We set $\mathbf{\Sigma} = 10 \cdot \mathbf{I}$, where $\mathbf{I}$ is the identity matrix.

### Formulation of Bayesian posterior distribution for the model parameters $\mathbf{\theta} = [r_{SS}, v_{0}, v_{1}, \alpha, \beta, w, \gamma]$ given the above information.


The prior is given by 
$$
\mathbb{P}(\theta) = \mathbb{P}(r_{SS})\mathbb{P}(v_{0})\mathbb{P}(v_{1})\mathbb{P}(\alpha)\mathbb{P}(\beta)\mathbb{P}(w)\mathbb{P}(\gamma) = \left\{
\begin{array}{ll}
      \frac{1}{22113} & 1.5 \leq r_{SS} \leq 4,\qquad 200 \leq v_{0} \leq 400, \qquad 550 \leq v_{1} \leq 950, \qquad 0.05 \leq \alpha \leq 0.5, \qquad 1\leq \beta \leq 1.75, \qquad 0.01 \leq w \leq 0.4, \qquad 0.06 \leq \gamma \leq 0.9 \\
      0 & \text{otherwise} \\
\end{array} 
\right. 
$$
since the uncertain parameters $r_{SS}, v_{0}, v_{1}, \alpha, \beta, w, \gamma$ are independent.

The posterior is given by (from Bayes' rule):
$$
\mathbb{P}(\theta | \mathbf{y}) = \frac{\mathbb{P}(\mathbf{y}|\theta) \mathbb{P}(\theta)}{\mathbb{P}(\mathbf{y})} = \frac{\mathbb{P}(\mathbf{y}|\theta) \mathbb{P}(\theta)}{\int_{\mathbb{R}^{7}} \mathbb{P}(\mathbf{y}|\theta)\mathbb{P}(\theta)\text{d}\mathbf{y}} \propto \mathbb{P}(\mathbf{y}|\theta) \mathbb{P}(\theta) 
$$


The likelihood function  updates our belief state by measuring the misfit between the data $\mathbf{y}$ and model predictions $\mathcal{M}(\mathbf{\theta})$ with a specific parameter set. Since we assume additive Gaussian measurement noise wth zero mean, i.e. $\eta_j  \in \mathcal{N}(\mathbf{\mu}= \mathbf{0},\mathbf{\Sigma})$, the measurements are $\mathbf{y}_{j}$ independent across observations, conditional on the parameters $\mathbf{\theta}$, i.e., $\mathbf{y}_{1}$ and $\mathbf{y}_{2}$ are independent given $\mathbf{\theta}_{1}$ and $\mathbf{\theta}_{2}$. Using the independence assumption, we can write the likelihood function as 
$$
\mathbb{P}(\mathbf{y}|\theta) =  \frac{1}{(2\pi)^{7/2} |\Sigma|^{1/2}} \exp \left( - \frac{1}{2}\left(\mathbf{y}_{i} - \mathcal{M}(\theta_{i})\right)^{\top} \mathbf{\Sigma}^{-1} \left(\mathbf{y}_{i} - \mathcal{M}(\theta_{i})\right)\right) \propto  \exp \left( -\frac{1}{2}\left(\mathbf{y}_{i} - \mathcal{M}(\theta_{i})\right)^{\top} \mathbf{\Sigma}^{-1} \left(\mathbf{y}_{i} - \mathcal{M}(\theta_{i})\right)\right).
$$

We initialize the walkers to start close to MAPs obtained from GSA for CR2048, CR2053, and CR2058 with QoI being the MAE.

### MCMC
MCMC is a procedure for generating a random walk in the parameter space that, over time, draws a representative set of samples from the distribution. Each point in a Markov chain $X(t_{i})$ depends only on the position of the previous step $X(t_{i−1})$. 

The MCMC Metropolis-Hastings (MH)  method involves simultaneously evolving an ensemble of $K$ walkers $S = {X_{k}}$ where the proposal distribution for one walker $k$ is based on the current positions of the $K − 1$ walkers in the complementary ensemble $S_{[k]} = \{X_{j} , \forall j\neq k\}$.


Here, we use a Python implemtation of the affine-invariant ensemble sampler proposed by **Goodman & Weare (2010)**.

In [4]:
reader = emcee.backends.HDFBackend("MCMC_results/CR2049.h5")

nburnin = 0
samples = reader.get_chain(flat=False, discard=nburnin)
samples_flatten  = reader.get_chain(flat=True, discard=nburnin)
log_prob_samples = reader.get_log_prob(flat=False, discard=nburnin)
log_prior_samples = reader.get_blobs(flat=False, discard=nburnin)
n_dim = 7
n_walkers = np.shape(samples)[1]

print("flat chain shape: {0}".format(samples.shape))
print("flat log prob shape: {0}".format(log_prob_samples.shape))

AttributeError: You must run the sampler with 'store == True' before accessing the results

### Acceptance ratio: 
The easiest and simplest indicator that things are going well is the acceptance fraction; it should be in the 0.2 to 0.5 range (there are theorems about this for specific problems; for example **Gelman, Roberts, & Gilks (1996)**).

In [ ]:
print("acceptance ratio = ", np.mean(reader.accepted/(reader.iteration - nburnin)))

### Log-likelihood in parameter space

In [ ]:
fig, ax = plt.subplots(figsize=(10, 2))
_ = ax.plot(log_prob_samples)
_ = ax.set_ylabel("Log-Likelihood")
_ = ax.set_xlabel("Number of iterations")

### A corner plot to show correlation (or lack thereof) of the seven model parameters.

In [ ]:
# list of labels used for plotting purposes. 
labels_list = np.array([r"$r_{SS}$", r"$v_{0}$", r"$v_{1}$", r"$\alpha$", r"$\beta$", r"$w$", r"$\gamma$"])

In [ ]:
fig = corner.corner(samples_flatten, labels=labels_list)


# Extract the axes
axes = np.array(fig.axes).reshape((n_dim, n_dim))

for ii in range(n_dim):
    ax = axes[ii, ii]
    ax.axvline(np.mean(samples[:, :, ii]), label="$\mu$", color="blue")
    ax.axvspan(np.mean(samples[:, :, ii]) - np.std(samples[:, :, ii]), 
               np.mean(samples[:, :, ii])  + np.std(samples[:, :, ii]), 
               alpha=0.5,
               color="lightblue", 
               label="$\mu \pm \sigma$")
    ax.set_title("$\mu = $" + str(round(np.mean(samples[:, :, ii]), 2))
                 +"\n $\sigma = $" + str(round(np.std(samples[:,:, ii]), 2)))
    ax.legend()
    
# Loop over the histograms
for yi in range(n_dim):
    for xi in range(n_dim):
        if xi < yi:
            ax = axes[yi, xi]
            ax.axvline(np.mean(samples[:, :, xi]), color="blue", ls="--")
            ax.axhline(np.mean(samples[:, :, yi]), color="blue", ls="--")
            ax.scatter(np.mean(samples[:, :, xi]), np.mean(samples[:, :, yi]), color="blue", s=10)

## Diagnostics for MCMC

**1) Mixing of the Chain (Heuristic):**
Good mixing occurs when more of the domain is explored. Bad mixing occurs when the step size is too large and most proposals are rejected, resulting in less space being explored. 

In [ ]:
fig, ax = plt.subplots(nrows=7,sharex=True, figsize=(5, 10))
for ii in range(7):
    ax[ii].plot(samples[:,:, ii], '-', alpha=0.7)
    ax[ii].set_ylabel(labels_list[ii])
ax[-1].set_xlabel("Number of samples")
ax[0].set_title("Walkers in Parameter Space")
plt.tight_layout()

**2) Autocorrelation:** Autocorrelation draws from the fundamental property of a Markov chain: *every Markov chain is memoryless*. We can compute the autocorrelation of states which are $\ell$ iterations apart as
$$
R(\ell) \equiv \frac{\sum_{i=1}^{n-\ell} (X_{i} -\bar{X})(X_{i+\ell} - \bar{X})}{\sum_{i=1}^{n} (X_{i} - \bar{X})^2}
$$
where $\bar{X}$ is the mean.

In [ ]:
fig, ax = plt.subplots(ncols=7, sharey=True, figsize=(15, 3))
for jj in range(7):
    for ii in np.arange(0, n_walkers, 1):
        ax[jj].plot(emcee.autocorr.function_1d(samples[:, ii, jj].T))
        ax[jj].set_xlabel("$\ell$")
        ax[jj].hlines(0, xmin=0, xmax=len(samples[:, 0, 0]), color="k", ls="--")
    ax[jj].set_title(str(labels_list[jj]))
ax[0].set_ylabel("$R(\ell)$")

plt.tight_layout()

**3) Integrated Autocorrelation (IAC) and Effective Sample Size (ESS):** In standard MC, the variance of the estimator is 
$$
\mathbb{V}\text{ar} \left( \frac{1}{N} \sum_{i=1}^{N} f(X^{(i)})\right) = \frac{\mathbb{V}\text{ar}(f(X))}{N}, \qquad X^{(i)} \sim i.i.d.
$$
In MCMC, samples are correlated, so we get additional covariance terms. If we assume that all $X_{i}$ and $X_{j}$ are samples from the target distribution, then we can approximate the variance of the MCMC estimator as 
$$
\mathbb{V}\text{ar}\left(\bar{f}(X)^{\text{MCMC}}\right) = \mathbb{V}\text{ar} \left( \frac{1}{N} \sum_{i=1}^{N} f(X^{(i)})\right) = \frac{\mathbb{V}\text{ar}\left(f(X) \right)}{N} \cdot \theta
$$
where the integrated autocorrelation (IAC) $\theta$ is defined as 
$$
\theta = \sum_{\tau=-\infty}^{\infty} \rho_{f}(\tau)
$$
where $\rho_{f}(\tau)$ is the normalized autocorrelation function of the stochastic process that generated the chain for $f$. You can estimate $\rho_{f}(\tau)$ using a finite chain $\{ f_{n}\}_{n=1}^{N}$ as 
$$
\hat{\rho}_{f}(\tau) = \hat{c}_{f}(\tau)/\hat{c}_{f}(0)
$$
where 
$$
\hat{c}_{f}(\tau) = \frac{1}{N-\tau} \sum_{n=1}^{N-\tau} (f_{n} - \mu_{f}) (f_{n+\tau} - \mu_{f})
$$
and 
$$
\mu_{f} = \frac{1}{N} \sum_{n=1}^{N} f_{n}.
$$
We stimate $\theta$ using the estimator for $\rho_{f}(\tau)$ as 
$$
\theta(M) = 1 + 2 \sum_{\tau=1}^{M} \hat{\rho}_{f}(\tau)
$$
The effective sample size of MCMC is then $\frac{N}{\theta}$ and we want $\frac{N}{\theta} \ll 1$. Essentially, $\theta$ is the number of steps that are needed fbefore the chain "forgets" where it started. Once we estimate $\theta$ then we can estimate the number of samples that we need to generate to reduce the relative error on the target inegral. Typically, parallel chains longer than $50\theta$ are often sufficient. 

The integrated autocorrelation time is an estimate of the number of steps needed in the chain in order to draw independent samples from the target density. A more efficient chain has a shorter autocorrelation time.

In [ ]:
reader.get_autocorr_time()
print("More than N=" + str(np.max(tau*50)) + " samples in each chain is sufficient.")